In [ ]:
#----------IMPORTS----------
from ultralytics import YOLO
import cv2
import numpy as np

from Libraries.sort import Sort

In [ ]:
#----------MODEL AND SORT LOAD----------
Model_Plates = YOLO('YOLOv8/best.pt')
Model_Number = YOLO('Number/best.pt')

#----------DICTIONARIES----------
id_to_plate = {}
id_to_plate_number = {}

#----------SORT AND SET----------
tracker = Sort()
detected_plates = set()

In [ ]:
#----------SCREEN RESIZE----------
def Resize_Screen(Frame):
    Frame_Width = int(Frame.get(cv2.CAP_PROP_FRAME_WIDTH))
    Frame_Height = int(Frame.get(cv2.CAP_PROP_FRAME_HEIGHT))

    Resized_Frame = (Frame_Width // 2, Frame_Height // 2, Frame_Width, Frame_Height)

    return Resized_Frame, Frame_Width

In [ ]:
#----------NUMBERS MODEL----------
def Plates_ID(Model_ID):
    if 0 <= Model_ID <= 9:
        return chr(ord('0') + Model_ID)
    elif 10 <= Model_ID <= 35:
        return chr(ord('A') + (Model_ID - 10))
    else:
        return ''
    
#----------PLATES MODEL ACTIVATION----------
def Plates_Model_Activation(Model_Data):
    Detection = []

    for Result in Model_Data[0].boxes:
        Class = int(Result.cls[0])
        if Result.xyxy[0].shape[0] >= 4:
            x1, y1, x2, y2 = Result.xyxy[0][:4].tolist()
            Score = Result.conf[0].item()

            if Class == 1:
                Detection.append([x1, y1, x2, y2, Score])
                
    return Detection

#----------NUMBERS MODEL ACTIVATION----------
def Numbers_Model_Activation(Model_Data, x1, y1, x2, y2):
    Detection = []

    for Result in Model_Data[0].boxes:
        Class = int(Result.cls[0])
        Character = Plates_ID(Class)
        Char_x1, Char_y1, Char_x2, Char_y2 = Result.xyxy[0][:4].tolist()

        if (Char_x1 >= 0 and Char_x2 <= (x2 - x1)) and (Char_y1 >= 0 and Char_y2 <= (y2 - y1)):
            Detection.append((Character, Char_x1))
    
    return Detection

In [ ]:
#----------PLATES TRACKER----------
def Plates_Tracker(Positions):
    Positions.sort(key=lambda x: x[1])
    Plate_Text = ''.join([c[0] for c in Positions])
    
    return Plate_Text

In [ ]:
#----------SCREEN SIZE----------
def Screen_Size(Frame):
    Frame_Resized = cv2.resize(Frame, (900, 500))
    cv2.imshow('COIMATRON', Frame_Resized)

    return Frame_Resized

#----------SCREEN OPTIONS----------
def Screen_Options(Pause, Frame):
    Key = cv2.waitKey(1) & 0xFF
    if Key == ord('z'):
        return True, False
    
    elif Key == ord('x'):
        return False, True
    
    if Pause:
        cv2.imshow('COIMATRON', Frame)
        while True:
            Key = cv2.waitKey(1) & 0xFF
            if Key == ord('z'):
                return False, False
            
            elif Key == ord('x'):
                return False, True
    
    return False, False

In [ ]:
# ----------CAMERA----------
cap = cv2.VideoCapture('IMG_1265.MOV')

# ----------PAUSE AND CLOSE STATE----------
is_paused = False
Closer = False

bottom_right_area, frame_width = Resize_Screen(cap)

current_plate_image = None

# ----------WHILE LOOP----------
while True:

    if not is_paused:
        _, frame = cap.read()
        bottom_right_frame = frame[bottom_right_area[1]:bottom_right_area[3], bottom_right_area[0]:bottom_right_area[2]]
        results = Model_Plates(frame)

        detections = Plates_Model_Activation(results)

        np_detections = np.array(detections)

        #----------TRACKER----------
        if np_detections.shape[0] > 0:
            tracked_objects = tracker.update(np_detections)

            # ----------READ NUMBERS----------
            tracked_ids = []
            for obj in tracked_objects:
                x1, y1, x2, y2, obj_id = obj
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                tracked_ids.append(obj_id)

                #----------VERIFY BOTTOM RIGHT----------
                if (x1 >= bottom_right_area[0] and y1 >= bottom_right_area[1]):
                    
                    if obj_id not in id_to_plate_number:
                        plate_area = frame[y1:y2, x1:x2]
                        current_plate_image = plate_area
                        
                        #----------ACTIVATE SECOND MODEL----------
                        number_results = Model_Number(plate_area)

                        char_positions = Numbers_Model_Activation(number_results, x1, y1, x2, y2)
                        id_to_plate_number[obj_id] = Plates_Tracker(char_positions)

                    #----------SHOW ID----------
                    plate_number = id_to_plate_number[obj_id]
                    text = f"ID: {int(obj_id)} Plate Number: {plate_number}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            if not any(obj_id in tracked_ids for obj_id in id_to_plate_number.keys()):
                current_plate_image = None

        if current_plate_image is not None:
            plate_resized = cv2.resize(current_plate_image, (200, 100))
            frame[0:100, frame_width - 200:frame_width] = plate_resized

        # ---------- SHOW CAMERA ----------
        frame_resized = Screen_Size(frame)
    
    is_paused, Closer = Screen_Options(is_paused, frame_resized)
    if Closer:
        break

cap.release()
cv2.destroyAllWindows()